# 00. Data Extraction (PDF Processing)

## 概要
本ノートブックでは、非構造化データである日本看護協会の調査PDFから、分析に必要な表データを抽出し、構造化データ（CSV）として保存します。

処理の負荷分散と保守性を高めるため、**「データの抽出（Extract）」**と**「データの加工・結合（Transform/Merge）」**の工程を分離しています。本ノートブックは抽出のみを担当します。

## 処理フロー
1. **Input**: `data/raw/` 配下のPDFファイルを読み込む
   - 対象ファイル：『日本看護協会_病院看護実態調査_離職率等_2025.pdf』
2. **Extraction**: `pdfplumber` を使用して特定のページ・表を抽出する
   - **表8**: 都道府県別 看護職員離職率
   - **表50**: 夜勤負担（72時間超）の状況
3. **Output**: CSV形式で `data/raw/` に保存する
   - 後続のノートブック（01_create_master_dataset）で読み込むためのRawデータとして出力

## 使用ライブラリ
- `pdfplumber`: PDFの表構造解析に使用
- `pandas`: データの整形・CSV出力に使用

In [1]:
import pdfplumber
import pandas as pd
from pathlib import Path

In [2]:
# ===============================
# Path Settings
# ===============================
BASE_DIR = Path("..")
RAW_DIR = BASE_DIR / "data" / "raw"
# PDFファイルのパス
pdf_path = RAW_DIR / "日本看護協会_病院看護実態調査_離職率等_2025.pdf"

# 抽出対象ページ（0-index）
P_TURNOVER = 73   # PDF 74ページ（表8）
P_NIGHT    = 108  # PDF 109ページ（表50）

In [3]:
# ===============================
# Functions
# ===============================
def extract_first_table(page_idx: int) -> pd.DataFrame:
    """指定ページの最初の表を抽出し、文字列としてDF化する"""
    with pdfplumber.open(pdf_path) as pdf:
        t = pdf.pages[page_idx].extract_tables()[0]
    df = pd.DataFrame(t).replace({None: ""})
    # mapで文字列化して空白除去
    df = df.map(lambda x: str(x).strip())
    return df

def pct_to_float(x: str):
    """'11.3%' -> 11.3 / '' -> NaN に変換"""
    if x is None:
        return pd.NA
    s = str(x).strip().replace("％", "%")
    if s == "":
        return pd.NA
    s = s.replace("%", "")
    try:
        return float(s)
    except:
        return pd.NA

In [4]:
# ===============================
# Main Processing
# ===============================

# 1) Turnover (表8) の抽出と整形
print("Extracting Turnover Data...")
df_t = extract_first_table(P_TURNOVER)
data_t = df_t.iloc[2:].copy() # ヘッダ除去

turnover = pd.DataFrame({
    "prefecture": data_t[0],
    "turnover_total": data_t[2],
    "turnover_new_grad": data_t[4],
    "turnover_experienced": data_t[6],
})
# 「計」を除外して数値変換
turnover = turnover[turnover["prefecture"] != "計"].copy()
for c in ["turnover_total", "turnover_new_grad", "turnover_experienced"]:
    turnover[c] = turnover[c].map(pct_to_float)


# 2) Night shift (表50) の抽出と整形
print("Extracting Night Shift Data...")
df_n = extract_first_table(P_NIGHT)
data_n = df_n.iloc[1:].copy() # ヘッダ除去

night = pd.DataFrame({
    "prefecture": data_n[0],
    "night_shift_72h_plus": data_n[5],
})
# 「計」を除外して数値変換
night = night[night["prefecture"] != "計"].copy()
night["night_shift_72h_plus"] = night["night_shift_72h_plus"].map(pct_to_float)

Extracting Turnover Data...
Extracting Night Shift Data...


In [5]:
# ===============================
# Save to CSV
# ===============================
out_turnover = RAW_DIR / "日本看護協会_離職率_都道府県別_2023.csv"
out_night = RAW_DIR / "日本看護協会_夜勤72h超過率_都道府県別_2024.csv"

turnover.to_csv(out_turnover, index=False, encoding="utf-8-sig")
night.to_csv(out_night, index=False, encoding="utf-8-sig")

print(f"✅ Saved: {out_turnover.name}")
print(f"✅ Saved: {out_night.name}")
display(turnover.head(3))
display(night.head(3))

✅ Saved: 日本看護協会_離職率_都道府県別_2023.csv
✅ Saved: 日本看護協会_夜勤72h超過率_都道府県別_2024.csv


,prefecture,turnover_total,turnover_new_grad,turnover_experienced
3,北海道,11.5,5.9,16.6
4,青森県,8.6,10.7,16.7
5,岩手県,6.8,7.8,19.1


,prefecture,night_shift_72h_plus
2,北海道,36.7
3,青森県,36.5
4,岩手県,11.8
